In [1]:
import numpy as np
import pandas as pd
import scipy.sparse
import xgboost as xgb
import mix_pandas as mix
import predict as predict_mix
import db_column_name as db

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 10, 7


cn = db.ColumnName()

target_minT = pd.read_csv('./data/31286_103.csv')
mix.set_index_date(target_minT, cn.date)

X = pd.read_csv('./data/character_31286.csv')
mix.set_index_date(X, cn.date)

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or 
       x == cn.offset]]
    
X = mix.mean_day(X)
target_minT.index = target_minT.index.round('D')

X = X.drop([cn.offset], axis=1)

target_minT = target_minT.reindex(X.index)
target_minT = mix.clean(target_minT)
X = X.reindex(target_minT.index)
X = mix.clean(X)

target_minT = mix.winsorized(target_minT, cn.value, [0.1, 0.80])
X = X.reindex(target_minT.index)
print(X.shape)

(1369, 54)


In [2]:
import pywt

def idwt(cA, cD):
    return  pywt.idwt(cA, cD, wavelet='db1', mode='constant')

cA, cD = pywt.dwt(target_minT.loc[:, cn.value].values, 
                  wavelet='db1', mode='constant')
# Approximation and detail coefficients.

cA, cD = pd.DataFrame(cA, columns=['c']), pd.DataFrame(cD, columns=['c'])

XA = mix.mean_pair(X)

x = []
size = X.shape[0]
for i in range(0, size, 2):
    if i + 1 < size:
            save = X.iloc[i:i + 2].diff(-1).fillna(0)
            save = save.iloc[0].values
            # iloc not include end in slice
            x.append(save)
    else:
        x.append(np.zeros(X.shape[1]))

XD = pd.DataFrame(x, columns=X.columns)

# XD = mix.diff_pair(X)

print(XA.shape)
print(XD.shape)

(685, 54)
(685, 54)


In [3]:
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

params = {
    'verbosity':0,
    'max_depth': 4,

#     'learning_rate': 0.05,
    'min_child_weight': 6,
#     'subsample':0.8, 
#     'colsample_bytree':0.8,
    'gamma': 12,
}

reg = xgb.XGBRegressor(**params)
predict_xa = predict_mix.predict_model_split(reg, XA, cA, 'c', 5)
predict_xd = predict_mix.predict_model_split(reg, XD, cD, 'c', 5)

for a, d in zip(predict_xa, predict_xd):
    # foo
    train_size = a[0].size
    test_size = a[1].size
    target_train = target_minT.iloc[0:train_size]
    target_test = target_minT.iloc[train_size - 1:train_size + test_size - 1]
    
    target_train['prediction'] = idwt(a[0].loc[:, 'prediction'], d[0].loc[:, 'prediction'])

    target_test['prediction'] = idwt(a[1].loc[:, 'prediction'], d[1].loc[:, 'prediction'])
    
    print("Train")
    predict_mix.print_mean(target_train[[cn.value]], target_train[['prediction']])
    print("Test")
    predict_mix.print_mean(target_test[[cn.value]], target_test[['prediction']])
    print()

c:\users\home1\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\home1\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Train
Mean squared error 1.42268
Mean absolute error 0.92321
Median absolute error 0.74099
Test
Mean squared error 18.66486
Mean absolute error 3.34746
Median absolute error 2.66813

Train
Mean squared error 1.41318
Mean absolute error 0.91714
Median absolute error 0.73250
Test
Mean squared error 15.99807
Mean absolute error 3.04902
Median absolute error 2.49369

Train
Mean squared error 1.51020
Mean absolute error 0.95816
Median absolute error 0.76012
Test
Mean squared error 42.82477
Mean absolute error 4.32072
Median absolute error 2.64075

Train
Mean squared error 1.75705
Mean absolute error 1.03062
Median absolute error 0.82287
Test
Mean squared error 52.09771
Mean absolute error 5.78596
Median absolute error 4.90323

Train
Mean squared error 1.91012
Mean absolute error 1.05861
Median absolute error 0.84887
Test
Mean squared error 18.32270
Mean absolute error 3.52418
Median absolute error 3.27311



c:\users\home1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\home1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
